In [5]:
import pandas as pd
import plotly.express as px

In [6]:
stn = 'g'

In [7]:
sm = pd.read_csv(f'./{stn}/records/sm')

batt = pd.read_csv(f'./{stn}/data/battery')
itemp = pd.read_csv(f'./{stn}/data/itemp')

sm.columns = sm.columns.str.strip()
batt.columns = batt.columns.str.strip()
itemp.columns = itemp.columns.str.strip()

In [99]:
sm.replace(65535, pd.NA, inplace=True)
sm.loc[:, sm.columns != 'Time'] = sm.loc[:, sm.columns != 'Time'].mask(sm.loc[:, sm.columns != 'Time'] > 30000, pd.NA)

print(f"SM1 range: {sm['SM1'].max()}, {sm['SM1'].min()}\nSM2 range: {sm['SM2'].max()}, {sm['SM2'].min()}\nSM3 range: {sm['SM3'].max()}, {sm['SM3'].min()}")
print(f"\nBattery Voltage range: {batt['Battery Voltage'].max()}, {batt['Battery Voltage'].min()}\nBattery Percentage range: {batt['Battery Percentage'].max()}, {batt['Battery Percentage'].min()}")
print(f"\nInternal Temperature range: {itemp['Internal Temperature'].max()}, {itemp['Internal Temperature'].min()}")
_ = input("Edit plotting and conversion parameters if necessary and press any key to continue...")

SM1 range: 16597.6, 220.8
SM2 range: 20957.0, 10517.2
SM3 range: 21873.8, 916.8

Battery Voltage range: 37.59436, 5.242043
Battery Percentage range: 3821.596, 226.8936

Internal Temperature range: -222.8594, -1480.234


In [120]:
ignore_nidx = 1
smn = ['SM1', 'SM2', 'SM3']
drop_sm = ['SM1', ]
smx = list(set(smn) ^ set(drop_sm))

sm_raw_to_vwc = {
    'SM1': [17_000, 200],
    'SM2': [21_000, 9_000],
    'SM3': [22_000, 10]
}

batt_v_factor = 2
batt_p_factor = -1.4

In [121]:
if drop_sm is not []:
    sm_filtered = sm[ignore_nidx:].copy().drop(columns=drop_sm)
else:
    sm_filtered = sm[ignore_nidx:].copy()

batt_filtered = batt[ignore_nidx:].copy().set_index('Time').reindex(sm_filtered['Time']).ffill().reset_index()
itemp_filtered = itemp[ignore_nidx:].copy().set_index('Time').reindex(sm_filtered['Time']).ffill().reset_index()

In [122]:
def convert_to_sm(raw_value, raw_dry, raw_sat):
    sm_value = (raw_value - raw_dry) / (raw_sat - raw_dry)
    return sm_value

station = sm_filtered.copy()
station['batt_v'] = batt_filtered['Battery Voltage']
station['batt_p'] = batt_filtered['Battery Percentage']
station['itemp'] = itemp_filtered['Internal Temperature']


for smidx in list(set(drop_sm) ^ set(smn)):
    station[[smidx]] = sm_filtered[[smidx]].map(lambda x: convert_to_sm(x, raw_dry=sm_raw_to_vwc[smidx][0], raw_sat=sm_raw_to_vwc[smidx][1]))

station['batt_v'] = station['batt_v'] * batt_v_factor
station['batt_p'] = station['batt_p'] * batt_p_factor

station['itemp'] = station['itemp'] + 273 + 25

cols = smx.copy()
cols.insert(0, 'Time')
station_plot_sm = station[cols].melt(id_vars=['Time'], value_name='SM', var_name='SMID')

In [123]:
fig = px.line(station_plot_sm, x='Time', y='SM', color='SMID', title=f'Station {stn.upper()}: Soil Moisture (VWC eqv., uncalibrated)')
fig.show()
fig.write_html(f'./vis/station_{stn}_sm.html')

In [104]:
fig = px.line(station, x='Time', y='batt_p', title=f'Station {stn.upper()}: Battery Percentage')
fig.update_traces(line=dict(color='cyan'))
fig.write_html(f'./vis/station_{stn}_batt.html')
fig.show()

In [105]:
fig = px.line(station, x='Time', y='itemp', title=f'Station {stn.upper()}: Internal Temperature')
fig.update_traces(line=dict(color='orange'))
fig.write_html(f'./vis/station_{stn}_itemp.html')
fig.show()